In [ ]:
import io
import ipywidgets as widgets
import matplotlib.pyplot as plt
import PIL.Image
import numpy as np
import urllib
from skimage.transform import resize
from matplotlib.image import imread
import os
from IPython.display import display
from skimage import io as io_url
import cv2
import numpy as np
from PIL import Image

In [ ]:
# Create image widgets
image3_spatial = widgets.Image(format='png', width=500, height=500, description='Spatial')
image3_freq    = widgets.Image(format='png', width=500, height=500, description='Frequency')
sidebyside     = widgets.HBox([image3_spatial, image3_freq])

# Create slider/select widgets
slider_inner   = widgets.FloatSlider(value=0,      min=0, max=1,      step=0.01, description='Inner radius')
slider_outer   = widgets.FloatSlider(value=1.44/2, min=0, max=1.44/2, step=0.01, description='Outer radius')

buf            = io.BytesIO()

orig_img = io_url.imread('https://img2.zergnet.com/2309662_300.jpg')
orig_img = np.mean(orig_img, -1)

x = np.fft.fftfreq(orig_img.shape[0]);
y = np.fft.fftfreq(orig_img.shape[1]);

xv, yv = np.meshgrid(x, y)
xv = np.fft.fftshift(xv)
yv = np.fft.fftshift(yv)

def filter_frequency(orig_img, mask):
  """
  You need to remove frequency based on the given mask.
  Params:
    orig_img: numpy image
    mask: same shape with orig_img indicating which frequency hold or remove
  Output:
    f_img: frequency image after applying mask
    img: image after applying mask
  """
    # Step 1: Transform using fft2
  frequency_coefs = np.fft.fft2(orig_img)

    # Step 2: Shift frequency coefs to center using fftshift
  shifted_coefs = np.fft.fftshift(frequency_coefs)

    # Step 3: Filter in frequency domain using the given mask
  filtered_coefs = shifted_coefs * mask

    # Step 4: Shift frequency coefs back using ifftshift
  inverse_shifted_coefs = np.fft.ifftshift(filtered_coefs)

    # Step 5: Invert transform using ifft2
  filtered_img = np.fft.ifft2(inverse_shifted_coefs)

    # Take the absolute value to get the real part (ignoring imaginary part)
  filtered_img = np.abs(filtered_img)

  return filtered_coefs , filtered_img

def on_value_change3(change):
    mask = (np.sqrt(xv**2 + yv**2) < slider_outer.value) & \
           (np.sqrt(xv**2 + yv**2) >= slider_inner.value)
    mask = np.float32(mask)

    fimg, img = filter_frequency(orig_img, mask)
    fimg = np.abs(fimg)  # Đảm bảo giá trị là dương
    fimg = fimg.astype(np.float64)  # Chuyển sang kiểu dữ liệu float64 để Pillow có thể xử lý

    buf.seek(0)
    tmp = PIL.Image.fromarray(255*img/(img.max()+0.0001))
    tmp = tmp.convert('L')
    tmp.save(buf, 'png')
    image3_spatial.value = buf.getvalue()

    buf.seek(0)
    tmp = PIL.Image.fromarray(255*np.log(0.0001*fimg + 1))
    tmp = tmp.convert('L')
    tmp.save(buf, 'png')
    image3_freq.value = buf.getvalue()


slider_inner.observe(on_value_change3, names='value')
slider_outer.observe(on_value_change3, names='value')

on_value_change3(0)

display(sidebyside)
display(slider_inner)
display(slider_outer)

FloatSlider(value=0.0, description='Inner radius', max=1.0, step=0.01)

FloatSlider(value=0.72, description='Outer radius', max=0.72, step=0.01)

In [ ]:
def read_img(img_path, img_size=(512, 512)):
  """
    + Đọc ảnh
    + Chuyển thành grayscale
    + Thay đổi kích thước ảnh thành img_size
  """
  try:
    img = cv2.imread(img_path, 0)
    img = cv2.resize(img, img_size)
    return img
  except Exception as e:
    print(str(e))


def create_hybrid_img(img1, img2, r):
  """
  Create hydrid image
  Params:
    img1: numpy image 1
    img2: numpy image 2
    r: radius that defines the filled circle of frequency of image 1. Refer to the homework title to know more.
  """
  # Step 1: Transform images using fft2
  img1_frequency_coefs = np.fft.fft2(img1)
  img2_frequency_coefs = np.fft.fft2(img2)

  # Step 2: Shift frequency coefs to center using fftshift
  img1_shifted_coefs = np.fft.fftshift(img1_frequency_coefs)
  img2_shifted_coefs = np.fft.fftshift(img2_frequency_coefs)

  # Step 3: Create a mask based on the given radius (r)
  mask = np.zeros(img1.shape)
  rows, cols = mask.shape
  center_row, center_col = rows // 2, cols // 2
  for i in range(rows):
      for j in range(cols):
          distance = np.sqrt((i - center_row) ** 2 + (j - center_col) ** 2)
          if distance <= r:
              mask[i, j] = 1

  # Step 4: Combine frequency of 2 images using the mask
  hybrid_shifted_coefs = img1_shifted_coefs * mask + img2_shifted_coefs * (1 - mask)

  # Step 5: Shift frequency coefs back using ifftshift
  hybrid_inverse_shifted_coefs = np.fft.ifftshift(hybrid_shifted_coefs)

  # Step 6: Invert transform using ifft2
  hybrid_img = np.fft.ifft2(hybrid_inverse_shifted_coefs)

  # Take the absolute value to get the real part (ignoring imaginary part)
  hybrid_img = np.abs(hybrid_img)

  return hybrid_img

  # You need to implement the function

In [ ]:
image_1_path = "https://img2.zergnet.com/2309662_300.jpg" # <-- need to change
image_2_path = "https://cdn.pixabay.com/photo/2012/12/27/19/40/chain-link-72864_1280.jpg" # <-- need to change
img_1 = read_img(image_1_path)
img_2 = read_img(image_2_path)
hybrid_img = create_hybrid_img(img_2, img_1, 14)
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(18, 15))
axes[0].imshow(img_1, cmap="gray")
axes[0].axis("off")
axes[1].imshow(img_2, cmap="gray")
axes[1].axis("off")
axes[2].imshow(hybrid_img, cmap="gray")
axes[2].axis("off")

OpenCV(4.8.0) /io/opencv/modules/imgproc/src/resize.cpp:4062: error: (-215:Assertion failed) !ssize.empty() in function 'resize'

OpenCV(4.8.0) /io/opencv/modules/imgproc/src/resize.cpp:4062: error: (-215:Assertion failed) !ssize.empty() in function 'resize'



IndexError: cannot do a non-empty take from an empty axes.